# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

API key found and looks good so far!


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [9]:
#method to construct messages for the ollama api
def messages_for(url):
    return [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(url)}
    ]
    

In [10]:
def select_relevant_links(url):
    messages = messages_for(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'social profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


{'links': [{'type': 'brand/about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'blog hub', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Zhihu', 'url': 'https://www.zhihu.com/org/huggingface'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'Blog post: ggml joins HF',
   'url': 'https://huggingface.co/blog/ggml-joins-hf'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-397B-A17B
Updated
about 3 hours ago
•
303k
•
923
nvidia/personaplex-7b-v1
Updated
8 days ago
•
539k
•
2.16k
Nanbeige/Nanbeige4.1-3B
Updated
1 day ago
•
178k
•
739
zai-org/GLM-5
Updated
10 days ago
•
179k
•
1.46k
MiniMaxAI/MiniMax-M2.5
Updated
7 days ago
•
209k
•
868
Browse 2M+ models
Spaces
Running
on
Zero
MCP
886
Wan2.2 14B Preview
🐌
886
generate a video from an image with a text prompt
Running
on
Zer

In [26]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 20 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\nabout 3 hours ago\n•\n303k\n•\n923\nnvidia/personaplex-7b-v1\nUpdated\n8 days ago\n•\n539k\n•\n2.16k\nNanbeige/Nanbeige4.1-3B\nUpdated\n1 day ago\n•\n178k\n•\n739\nzai-org/GLM-5\nUpdated\n10 days ago\n•\n179k\n•\n1.46k\nMiniMaxAI/MiniM

In [27]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a vibrant AI community and collaboration platform dedicated to building the future of machine learning. Founded in 2016 and headquartered in Paris, France, Hugging Face empowers machine learning engineers, scientists, and enthusiasts to create, share, and experiment with open-source machine learning models, datasets, and applications.

At its core, Hugging Face aims to **solve and democratize artificial intelligence through natural language processing (NLP)** and deep learning technologies. With a fast-growing global community and some of the most widely used open-source ML libraries, Hugging Face is firmly at the heart of the AI revolution.

---

## What Hugging Face Offers

- **Models & Datasets:** Access over 2 million open-source ML models and 500,000+ datasets spanning multiple modalities, including text, image, video, audio, and 3D data.
- **Spaces:** Host and deploy your own ML applications or explore countless community-built AI apps.
- **Community Hub:** Collaborate, discover, share, and build your machine learning profile by contributing to the ML ecosystem.
- **Enterprise Solutions:** Accelerate machine learning deployments with faster compute options and scalable production-grade inference.
- **Open Source Stack:** Fully embrace the HF open-source ecosystem for rapid experimentation and production readiness.

---

## Company Culture

Hugging Face fosters a **collaborative and inclusive environment** driven by innovation and community engagement. The company believes in:

- **Open and Ethical AI:** Building AI for the common good through transparency and sharing.
- **Community Empowerment:** Supporting the next generation of ML engineers and scientists.
- **Passion for Technology:** Constantly pushing the boundaries of AI research and development.
- **Continuous Learning:** Providing robust documentation, forums, and a blog for knowledge sharing.

---

## Hugging Face Community & Customers

The Hugging Face platform is home to:

- A **global community** of ML practitioners, researchers, and developers.
- Organizations relying on its tools, from startups to large enterprises, to build and deploy AI solutions efficiently.
- Developers contributing to and utilizing libraries for natural language processing, computer vision, and more.

---

## Careers at Hugging Face

As a growing company with **51-200 employees**, Hugging Face is actively seeking talented individuals passionate about AI, machine learning, and open source. Their team includes scientists, engineers, and thought leaders pioneering the future of AI.

Current job openings span multiple roles in software development, research, and enterprise support. Employees enjoy a dynamic and mission-driven workplace focused on making AI accessible and impactful.

Explore career opportunities: [Hugging Face Careers](https://huggingface.co)

---

## Get Started With Hugging Face

- Join over hundreds of thousands of users exploring models and apps.
- Share your own models and datasets on the Hugging Face Hub.
- Experiment with AI chatbots like **HuggingChat Omni**.
- Access tools and paid compute to accelerate your ML workloads.

Visit: [https://huggingface.co](https://huggingface.co)

---

**Hugging Face — The AI community building the future.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

## About Hugging Face

Hugging Face is the AI community building the future. It is a leading platform where the machine learning community collaborates on models, datasets, and applications. The company supports an open-source ecosystem that fosters collaboration in developing and sharing machine learning (ML) resources.

## Platform Features

- **Extensive Model Repository**: Browse over 2 million ML models covering various use cases and modalities, including text, image, video, audio, and even 3D data.
- **Datasets**: Access over 500,000 datasets for training, benchmarking, and experimentation.
- **Spaces**: Explore and host ML-powered applications and demos, such as text-to-speech generation, image editing, and video creation tools.
- **Community & Collaboration**: Create, discover, and collaborate on ML projects to accelerate innovation.
- **Open Source Stack**: Benefit from the Hugging Face open source tools and libraries that support faster and more efficient ML workflows.
- **Enterprise Solutions**: Designed to meet company needs with scalable infrastructure and paid compute resources.

## Company Culture

Hugging Face fosters a vibrant and active AI & ML enthusiast community with over 83,000 followers engaging in continuous knowledge sharing and co-development. The culture promotes openness, innovation, and collaboration among developers, researchers, and enterprises alike.

## Customers

The platform serves a global audience of machine learning practitioners, researchers, developers, and enterprises seeking to leverage state-of-the-art models and datasets. Its users range from individual AI enthusiasts building portfolios to large organizations requiring robust, scalable AI solutions.

## Careers & Opportunities

Hugging Face is an exciting place for individuals passionate about AI and open source. The company values contributors who thrive in a collaborative environment and are eager to push the boundaries of machine learning technology. Career opportunities often focus on engineering, research, community engagement, and enterprise support roles.

## Join the AI Revolution with Hugging Face

- Explore trending models like **Qwen3.5-397B**, **personaplex-7b-v1**, and others.
- Host and discover cutting-edge AI applications in various domains.
- Build your ML portfolio by sharing your work with a global community.
- Access enterprise-grade compute and tools to accelerate AI projects.
- Be part of a growing collaborative ecosystem aiming to shape the future of artificial intelligence.

**Website:** https://huggingface.co  
**Get Started:** Sign up today to join the thriving AI community and transform your machine learning journey!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>